<a href="https://colab.research.google.com/github/xmendevs/HNG-i13-Interns-Analytics/blob/main/Hng_Intern_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cell 0 — Install packages

In [ ]:
# Colab Notebook: HNG i13 — full runnable pipeline
# ===============================
# 0) Install libs (run once)
!pip install -q lightgbm optuna shap ydata-profiling scikit-learn pandas numpy matplotlib seaborn joblib

# -------------------------
# Imports & settings
# -------------------------
import os, warnings
warnings.filterwarnings('ignore')
import pandas as pd, numpy as np
from pathlib import Path
import matplotlib.pyplot as plt, seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import lightgbm as lgb
import joblib, optuna, shap
from ydata_profiling import ProfileReport

# Settings
DATA_DIR = '/content'
MASTER_CSV = os.path.join(DATA_DIR, 'master_cleanedd.csv')
CHECKIN_CSV = os.path.join(DATA_DIR, 'checkin.csv')
OUT_DIR = Path('/content/powerbi_outputs')
OUT_DIR.mkdir(parents=True, exist_ok=True)

OPTUNA_TRIALS = 25
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
print("Outputs folder:", OUT_DIR)


Outputs folder: /content/powerbi_outputs


In [ ]:
# 1) Load & normalize
master = pd.read_csv(MASTER_CSV)
checkin = pd.read_csv(CHECKIN_CSV)

# Normalize column names (snake_case)
def norm_cols(df):
    df.columns = [c.strip().lower().replace(' ', '_') for c in df.columns]
    return df

master = norm_cols(master)
checkin = norm_cols(checkin)

print("Master shape:", master.shape)
print("Checkin shape:", checkin.shape)
print("Master columns:", master.columns.tolist()[:40])


Master shape: (1411, 9)
Checkin shape: (12, 7)
Master columns: ['series', 'slackid', 'email', 'stage_1', 'stage_2', 'stage_3', 'stage_4', 'stage_7', 'track']


In [ ]:
# 2) Ensure 'series' exists and use it as intern id
if 'series' not in master.columns:
    raise ValueError("Column 'series' not found in master_cleanedd.csv; please provide it as intern unique id.")
master['series'] = master['series'].astype(str)

# Quick view
master[['series'] + [c for c in master.columns if c.startswith('stage')][:8]].head()


,series,stage_1,stage_2,stage_3,stage_4,stage_7
0,1462,6.5,NaN,NaN,NaN,NaN
1,940,3.5,6.5,6.5,8.3,NaN
2,1099,4.0,5.0,NaN,8.3,NaN
3,466,5.7,NaN,NaN,NaN,NaN
4,1031,4.0,5.0,7.5,8.2,NaN


In [ ]:
# 3) Basic feature engineering: stages_completed, total_score, mean_score, last_stage_score, is_active
stage_cols = sorted([c for c in master.columns if c.startswith('stage')], key=lambda s: int(''.join([ch for ch in s if ch.isdigit()]) or 0))
print("Detected stage columns:", stage_cols)

# stages_completed (count non-null stage scores)
master['stages_completed'] = master[stage_cols].notna().sum(axis=1)

# total and mean (fillna 0 for sum only)
master['total_score'] = master[stage_cols].fillna(0).sum(axis=1)
master['mean_score'] = master[stage_cols].mean(axis=1)

# last non-null stage score
def last_non_null(row):
    vals = row[stage_cols].dropna()
    return vals.iloc[-1] if len(vals)>0 else np.nan
master['last_stage_score'] = master.apply(last_non_null, axis=1)

# is_active if stage_7 present
master['is_active'] = master['stage_7'].notna().astype(int)

# quick sanity
master[['series','stages_completed','total_score','mean_score','last_stage_score','is_active']].head()


Detected stage columns: ['stage_1', 'stage_2', 'stage_3', 'stage_4', 'stage_7']


,series,stages_completed,total_score,mean_score,last_stage_score,is_active
0,1462,1,6.5,6.500000,6.5,0
1,940,4,24.8,6.200000,8.3,0
2,1099,3,17.3,5.766667,8.3,0
3,466,1,5.7,5.700000,5.7,0
4,1031,4,24.7,6.175000,8.2,0


In [ ]:
# 4) Save a small EDA profile (optional, may be large)
try:
    profile = ProfileReport(master.sample(min(len(master), 1000), random_state=RANDOM_STATE), title="Master sample profile", minimal=True)
    profile.to_file(OUT_DIR / "master_profile_report.html")
    print("Saved master_profile_report.html")
except Exception as e:
    print("Profile generation skipped (heavy) or failed:", e)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 14/14 [00:00<00:00, 36.56it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Saved master_profile_report.html


Classification: predict is_active (has stage7)

In [ ]:
# 5) Classification dataset (is_active)
clf_features = ['track', 'stages_completed', 'mean_score', 'last_stage_score', 'total_score']
clf_features = [c for c in clf_features if c in master.columns]
df_clf = master[clf_features + ['is_active','series']].copy()

# simple imputations
for c in df_clf.select_dtypes(include='number').columns:
    df_clf[c] = df_clf[c].fillna(df_clf[c].median())
for c in df_clf.select_dtypes(include='object').columns:
    df_clf[c] = df_clf[c].fillna('Unknown')

X_clf = df_clf[clf_features]
y_clf = df_clf['is_active']

X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X_clf, y_clf, test_size=0.2, stratify=y_clf, random_state=RANDOM_STATE)
numeric_feats = [c for c in X_train_clf.columns if X_train_clf[c].dtype != 'object']
categorical_feats = [c for c in X_train_clf.columns if X_train_clf[c].dtype == 'object']

preprocessor_clf = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numeric_feats),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_feats)
], remainder='drop')

baseline_clf = Pipeline([('pre', preprocessor_clf), ('clf', lgb.LGBMClassifier(n_estimators=300, random_state=RANDOM_STATE))])
baseline_clf.fit(X_train_clf, y_train_clf)

y_proba = baseline_clf.predict_proba(X_test_clf)[:,1]
y_pred = baseline_clf.predict(X_test_clf)
print("Classification AUC (baseline):", roc_auc_score(y_test_clf, y_proba))
print(classification_report(y_test_clf, y_pred))


[LightGBM] [Info] Number of positive: 157, number of negative: 971
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001138 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 575
[LightGBM] [Info] Number of data points in the train set: 1128, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.139184 -> initscore=-1.822081
[LightGBM] [Info] Start training from score -1.822081
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

Regression: predict actual stage_7 (for rows where stage_7 exists) — used for evaluation

In [ ]:
# 6) Regression to predict stage_7 (train only on rows with stage_7)
df_reg = master.dropna(subset=['stage_7']).copy()
regress_features = ['track', 'stages_completed', 'mean_score', 'last_stage_score', 'total_score']
regress_features = [c for c in regress_features if c in df_reg.columns]

# prepare X,y
Xr = df_reg[regress_features]
yr = df_reg['stage_7']

# impute
for c in Xr.select_dtypes(include='number').columns:
    Xr[c] = Xr[c].fillna(Xr[c].median())
for c in Xr.select_dtypes(include='object').columns:
    Xr[c] = Xr[c].fillna('Unknown')

X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(Xr, yr, test_size=0.25, random_state=RANDOM_STATE)

preprocessor_reg = ColumnTransformer(transformers=[
    ('num', StandardScaler(), [c for c in Xr.columns if Xr[c].dtype != 'object']),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), [c for c in Xr.columns if Xr[c].dtype == 'object'])
], remainder='drop')

reg_model = Pipeline([('pre', preprocessor_reg), ('clf', lgb.LGBMRegressor(n_estimators=800, learning_rate=0.03, random_state=RANDOM_STATE))])
reg_model.fit(X_train_r, y_train_r)
pred_r = reg_model.predict(X_test_r)

print("Stage7 regression metrics:")
print("MAE:", mean_absolute_error(y_test_r, pred_r))

# FIXED RMSE
rmse = mean_squared_error(y_test_r, pred_r) ** 0.5
print("RMSE:", rmse)

print("R2:", r2_score(y_test_r, pred_r))



[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000043 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 134
[LightGBM] [Info] Number of data points in the train set: 147, number of used features: 7
[LightGBM] [Info] Start training from score 6.877551
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

Forecast: train a future model (using stages 1–6 → stage_7) and then forecast Stage 8

In [ ]:
# 7) Future stage forecasting model: use available stage_1..stage_6 to predict stage_7, then use that model to forecast stage_8.
# Create X_future from stage_1..stage_6 if present + track + aggregate features
stage_inputs = [c for c in master.columns if c.startswith('stage_') and any(ch.isdigit() for ch in c)]
# choose up to stage_6 only (exclude stage_7)
stage_inputs = [c for c in stage_inputs if int(''.join([ch for ch in c if ch.isdigit()])) <= 6]
print("Stage inputs for future model:", stage_inputs)

future_features = ['track'] + stage_inputs + ['stages_completed','mean_score','total_score']
future_features = [c for c in future_features if c in master.columns]

# Train on rows with stage_7 (we know real stage_7)
df_future_train = master.dropna(subset=['stage_7']).copy()
Xf = df_future_train[future_features].copy()
yf = df_future_train['stage_7'].copy()

# impute
for c in Xf.select_dtypes(include='number').columns:
    Xf[c] = Xf[c].fillna(Xf[c].median())
for c in Xf.select_dtypes(include='object').columns:
    Xf[c] = Xf[c].fillna('Unknown')

Xf_train, Xf_test, yf_train, yf_test = train_test_split(Xf, yf, test_size=0.25, random_state=RANDOM_STATE)

preprocessor_fut = ColumnTransformer(transformers=[
    ('num', StandardScaler(), [c for c in Xf.columns if Xf[c].dtype != 'object']),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), [c for c in Xf.columns if Xf[c].dtype == 'object'])
], remainder='drop')

future_model = Pipeline([('pre', preprocessor_fut), ('clf', lgb.LGBMRegressor(n_estimators=1000, learning_rate=0.02, random_state=RANDOM_STATE))])
future_model.fit(Xf_train, yf_train)
pred_fut_test = future_model.predict(Xf_test)
print("Future model (stage7-as-proxy) MAE:", mean_absolute_error(yf_test, pred_fut_test))


Stage inputs for future model: ['stage_1', 'stage_2', 'stage_3', 'stage_4']
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000056 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 185
[LightGBM] [Info] Number of data points in the train set: 147, number of used features: 10
[LightGBM] [Info] Start training from score 6.877551
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [ ]:
# 8) Predict stage7 for all rows that have stage_7 (evaluation) and predict stage8 (forecast) for those with stage_7
# For consistency we'll produce predicted_stage7 (from reg_model) and predicted_stage8 (from future_model)
# Prepare X_full_reg for regression model (train-on-stage7 features)
X_full_reg = master[regress_features].copy()
for c in X_full_reg.select_dtypes(include='number').columns:
    X_full_reg[c] = X_full_reg[c].fillna(X_full_reg[c].median())
for c in X_full_reg.select_dtypes(include='object').columns:
    X_full_reg[c] = X_full_reg[c].fillna('Unknown')

# predicted stage7 (model trained above)
pred_stage7_all = reg_model.predict(X_full_reg)
master['predicted_stage7_score'] = pred_stage7_all

# For stage8 forecast: use future_features (stages 1-6 + others)
X_full_future = master[future_features].copy()
for c in X_full_future.select_dtypes(include='number').columns:
    X_full_future[c] = X_full_future[c].fillna(X_full_future[c].median())
for c in X_full_future.select_dtypes(include='object').columns:
    X_full_future[c] = X_full_future[c].fillna('Unknown')

# predicted stage8 (forecast). We only meaningfully interpret this for interns with stage_7 present
pred_stage8_all = future_model.predict(X_full_future)
master['predicted_stage8_score'] = pred_stage8_all

# For interns with real stage_7, calculate residual of predicted_stage7 vs actual for evaluation
master['stage7_residual'] = np.nan
mask_stage7 = master['stage_7'].notna()
master.loc[mask_stage7, 'stage7_residual'] = master.loc[mask_stage7, 'stage_7'] - master.loc[mask_stage7, 'predicted_stage7_score']


SHAP explainability (global + per-intern local) for the future_model (Stage8 predictions)

In [ ]:
# 9) SHAP (for future_model — may be memory heavy)
try:
    # Transform training data to get feature names
    pre = future_model.named_steps['pre']
    # numeric names
    num_names = [c for c in Xf.columns if Xf[c].dtype != 'object']
    cat_names = []
    if len([c for c in Xf.columns if Xf[c].dtype == 'object'])>0:
        cat_encoder = pre.named_transformers_['cat']
        try:
            cat_names = list(cat_encoder.get_feature_names_out([c for c in Xf.columns if Xf[c].dtype == 'object']))
        except:
            # fallback: create combined names
            cat_names = []
    feature_names = num_names + cat_names

    # sample to limit memory
    pre_transform_all = future_model.named_steps['pre'].transform(X_full_future)
    sample_n = min(500, pre_transform_all.shape[0])
    sample_idx = np.random.choice(pre_transform_all.shape[0], sample_n, replace=False)
    X_shap = pre_transform_all[sample_idx]

    explainer = shap.TreeExplainer(future_model.named_steps['clf'])
    raw_shap = explainer.shap_values(X_shap)
    if isinstance(raw_shap, list):
        shap_vals = raw_shap[0]
    else:
        shap_vals = raw_shap

    mean_abs = np.abs(shap_vals).mean(axis=0)
    shap_df = pd.DataFrame({'feature': feature_names, 'mean_abs_shap': mean_abs}).sort_values('mean_abs_shap', ascending=False)
    shap_df.to_csv(OUT_DIR / 'shap_feature_importance.csv', index=False)
    print("Saved SHAP global importance")
    # local (wide) shap for all rows (may be large)
    raw_all = explainer.shap_values(pre_transform_all)
    if isinstance(raw_all, list):
        raw_all = raw_all[0]
    shap_local_df = pd.DataFrame(raw_all, columns=feature_names)
    # attach series (intern id)
    shap_local_df.insert(0, 'series', master['series'].astype(str).values)
    shap_local_df.to_csv(OUT_DIR / 'shap_local_sample.csv', index=False)
    print("Saved SHAP local sample (wide)")
except Exception as e:
    print("SHAP step skipped or failed:", e)
    # produce empty placeholders so Power BI imports do not fail
    pd.DataFrame().to_csv(OUT_DIR / 'shap_feature_importance.csv', index=False)
    pd.DataFrame().to_csv(OUT_DIR / 'shap_local_sample.csv', index=False)


Saved SHAP global importance
Saved SHAP local sample (wide)


Final exports for Power BI

In [ ]:
# 10) Export outputs for Power BI
# Export master with predictions and series (intern id)
master.to_csv(OUT_DIR / 'powerbi_master_scores.csv', index=False)
# Export stage7 predictions (evaluation subset)
master.loc[mask_stage7, ['series','stage_7','predicted_stage7_score','stage7_residual']].to_csv(OUT_DIR / 'powerbi_stage7_predictions.csv', index=False)
# Export stage8 forecast (full export)
master[['series','predicted_stage8_score','predicted_stage7_score','stage_7','stage7_residual']].to_csv(OUT_DIR / 'powerbi_stage8_predictions.csv', index=False)
# Export checkin wide & long
checkin.to_csv(OUT_DIR / 'powerbi_checkin_raw.csv', index=False)
if 'checkin_long' in globals():
    checkin_long.to_csv(OUT_DIR / 'powerbi_checkin_long.csv', index=False)
print("Saved Power BI CSVs to", OUT_DIR)


Saved Power BI CSVs to /content/powerbi_outputs


In [ ]:
# 10) Export outputs for Power BI
# Export master with predictions and series (intern id)
master.to_csv(OUT_DIR / 'powerbi_master_scores.csv', index=False)
# Export stage7 predictions (evaluation subset)
master.loc[mask_stage7, ['series','stage_7','predicted_stage7_score','stage7_residual']].to_csv(OUT_DIR / 'powerbi_stage7_predictions.csv', index=False)
# Export stage8 forecast (full export)
master[['series','predicted_stage8_score','predicted_stage7_score','stage_7','stage7_residual']].to_csv(OUT_DIR / 'powerbi_stage8_predictions.csv', index=False)
# Export checkin wide & long
checkin.to_csv(OUT_DIR / 'powerbi_checkin_raw.csv', index=False)
if 'checkin_long' in globals():
    checkin_long.to_csv(OUT_DIR / 'powerbi_checkin_long.csv', index=False)
print("Saved Power BI CSVs to", OUT_DIR)


Saved Power BI CSVs to /content/powerbi_outputs
